<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Introduction aux SVM en R

** Objectifs : ** 

- Apprendre à manipuler les SVM pour la classification en R avec la librairie **`kernlab`**
- Observer l'effet du noyau et de la pénalité sur les règles de classification

# SVM linéaires :

## Génération des données

On commence par générer un jeu de données "jouet" : deux échantillons gaussiens en dimension 2 pour la visualisation avce des moyennes de signes différents.

In [ ]:
# Generation  des variables explicatives :

n=80
p=2
s=1
m1=0
m2=3

x1=matrix(rnorm(n*p,mean=m1,sd=s),n,p)
x2=matrix(rnorm(n*p,mean=m2,sd=s),n,p)
x=rbind(x1,x2)
dim(x)

In [ ]:
# Génération des labels
y=matrix(c(rep(-1,n),rep(1,n)))

In [ ]:
# Visualisation des données
plot(x,col=ifelse(y>0,1,2))
legend("topleft", c('y=1','y=-1'),col=c(1,2),pch=1, text.col=c(1,2))

In [ ]:
# Echantillon d'apprentissage et échantillon test
#ntrain=round(1.5*n)
index=sample(2*n,round(1.5*n))
xtrain = x[index,]
xtest= x[-index,]
ytrain=y[index]
ytest=y[-index]


## Apprentissage d'un SVM linéaire

In [ ]:
library(kernlab)
help(ksvm)
linsvm=ksvm(xtrain, ytrain,type="C-svc",kernel='vanilladot',C=100)
summary(linsvm)
linsvm

In [ ]:
attributes(linsvm)

In [ ]:
alphaindex(linsvm)

In [ ]:
alpha(linsvm)

In [ ]:
b(linsvm)

In [ ]:
plot(linsvm,data=xtrain)

## Prédiction 

In [ ]:
ypred=predict(linsvm,xtest)
ypred
table(ytest,ypred)

In [ ]:
# Taux de bonnes prédictions
sum(ypred==ytest)/length(ytest)

In [ ]:
ypredscore=predict(linsvm,xtest,type="decision")
ypredscore

** Tracé des courbes ROC **

In [ ]:
library(ROCR)

In [ ]:
pred=prediction(ypredscore,ytest)

In [ ]:
pred

In [ ]:
# Courbe ROC :
res1=performance(pred,measure="tpr",x.measure="fpr")
plot(res1)

In [ ]:
# Performance en fonction du seuil
res3=performance(pred,measure="acc")
plot(res3)


##  Estimation de l'erreur par validation croisée

In [ ]:
linsvm=ksvm(x, y,type="C-svc",kernel='vanilladot',C=1,cross=5)
summary(linsvm)
linsvm

In [ ]:
print(cross(linsvm))

In [ ]:
plot(linsvm,data=x)

## Optimisation du paramètre $C$

In [ ]:
linsvm=ksvm(x, y,type="C-svc",kernel='vanilladot',C=10,cross=5)
summary(linsvm)
linsvm

In [ ]:
plot(linsvm,data=x)

**Q.** Que constatez-vous sur l'erreur estimée par validation croisée ? Comment expliquez vous ce phénomène ? 

**Q.** Ecrire un programme qui permet de tracer en fonction de $C$ l'erreur estimée par validation croisée. Refaire ce tracé avec des données moins nettement séparées (par exemple en prenant $m1=1$). 

# SVM non linéaires

## Génération des données : 

On génère des données qui ne seront pas linéairement séparables : pour cela, chacun des deux jeux de données est généré selon un mélange gaussien : 

In [ ]:
# Generation  des variables explicatives
n=40
p=2
s=1
m1=0
m2=3

x11=matrix(rnorm(n*p,mean=m1,sd=s),n,p)

x12=matrix(c(rnorm(n,mean=m1,sd=s),rnorm(n,mean=m2,sd=s)),n,p)

x21=matrix(c(rnorm(n,mean=m2,sd=s),rnorm(n,mean=m1,sd=s)),n,p)

x22=matrix(rnorm(n*p,mean=m2,sd=s),n,p)

x=rbind(x11,x22,x12,x21)
dim(x)


In [ ]:
# Génération des labels
y=matrix(c(rep(-1,2*n),rep(1,2*n)))


In [ ]:
# Visualisation des données
plot(x,col=ifelse(y>0,1,2))
legend("topleft", c('y=1','y=-1'),col=c(1,2),pch=1, text.col=c(1,2))

Au lieu d'utiliser un noyau linéaire, nous allons utiliser un noyau gaussien. 

In [ ]:
gaussvm=ksvm(x,y,type="C-svc",kernel='rbf',kpar=list(sigma=1),C=10^10,cross=5)
plot(gaussvm, data=x)
print(gaussvm)

In [ ]:
# Selection automatique de sigma
gaussvm=ksvm(x,y,type="C-svc",kernel='rbf',C=1,cross=5)
plot(gaussvm, data=x)
print(gaussvm)

**Q. ** Tracer en fonction de $C$ :
- l'erreur d'apprentissage, l'erreur estimée par validation croisée 
- le nombre de support vectors retenus. 

** Q.** Tester la méthode avec d'autres noyaux et voyez l'importance du choix du noyau ! 


# Application à des données génomiques

A data frame with 106 observations and 58 variables. The first variableClass
is a factor with levels ** + ** for a promoter gene and ** - **
for a non-promoter gene.  The remaining 57 variables V2 to V58 are factors describing the sequence. 
The DNA bases are coded as follows:
- a : adenine
- c : cytosine
- g : guanine
- t: thymine

Réf. : Powell, G., Shavlik, J. and Noordewier, M.
Refinement of Approximate Domain Theories by Knowledge-Based Artificial Neural Networks.
In Proceedings of the Eighth National Conference on Artificial Intelligence (AAAI-90)

In [ ]:
data(promotergene)

In [ ]:
summary(promotergene)

In [ ]:
model=ksvm(Class~.,data=promotergene,kernel="laplacedot", kpar="automatic",C=5,cross=4)
print(model)

In [ ]:
x=promotergene[,-1]
y=promotergene$Class


In [ ]:
Cpred=predict(model,x)

table(Cpred,y)
# Attention, on ne voit ici que l'erreur apparente ! Regarder l'erreur estimée par cross-validation

# Classification multi-classes

Pour la classification multiclasses ($k>2$), la fonction **`ksvm`** utilise une approche "one-againts one"", dans laquelle $k(k-1)/2$ classifieurs binaires sont entrainés. La classe est attribuée par un vote majoritaire. 

D'autres algorithmes utilisent une approche "one versus all", où on teste à chaque fois une classe contre toutes les autres. On a alors $k$ classifieurs à construire et on retient généralement la classe qui permet d'avoir la plus grande marge. 

In [ ]:
# Generation  des variables explicatives
n=40
p=2
s=1
m1=0
m2=3

x11=matrix(rnorm(n*p,mean=m1,sd=s),n,p)

x12=matrix(c(rnorm(n,mean=m1,sd=s),rnorm(n,mean=m2,sd=s)),n,p)

x21=matrix(c(rnorm(n,mean=m2,sd=s),rnorm(n,mean=m1,sd=s)),n,p)

x22=matrix(rnorm(n*p,mean=m2,sd=s),n,p)

x=rbind(x11,x22,x12,x21)
dim(x)

In [ ]:
# Génération des labels
n=40
y=matrix(c(rep(1,n),rep(2,n),rep(3,n),rep(4,n)))


In [ ]:
# Visualisation des données
plot(x,col=y)
legend("topleft", c('y=1','y=2',"y=3","y=4"),col=c(1,2,3,4),pch=1, text.col=c(1,2,3,4))

In [ ]:

model <- ksvm(x,y,type="C-svc",kernel="rbfdot",C=10,prob.model=TRUE,cross=5)
model
## get fitted values
fitted(model)
## Test on the training set with probabilities as output
#ypred=predict(irismodel, x, type="probabilities")
ypred=predict(model, x)
table(y,ypred)


In [ ]:

data(iris)
## Create a kernel function using the build in rbfdot function
rbf <- rbfdot(sigma=0.1)
rbf
## train a bound constraint support vector machine
irismodel <- ksvm(Species~.,data=iris,type="C-bsvc",kernel=rbf,C=10,prob.model=TRUE,cross=5)
irismodel
## get fitted values
irispred=fitted(irismodel)

table(irispred,iris$Species)
## Test on the training set with probabilities as output
#predict(irismodel, iris[,-5], type="probabilities")